# File viewer
This notebook was created to explore all the CSV files in this project

## Variables
**above_avg_threshold:** this variable it's used on the secondary plot to highlight those cells which are greater than the mean in a certain frame by this threshold, so if the mean of a frame is 22 ℃ all the cells that are greater than (22 + **above_avg_threshold**) will be highligted in white from the rest, when this variable it's lowered it increases the sensitivity of the plot.
    

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import timedelta
import matplotlib.pylab as plt
import os
import ipywidgets as widgets
from ipywidgets import *

#Constants
fall_scene_tags_headers = ['filename','scene','event']

csv_headers = ['timestamp','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16']
data_headers = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16']
fall_directory = './fall_data/'
non_fall_directory = './non_fall_data/'

#Variables
above_avg_threshold = 0.6



In [2]:
# Get a dataframe from a CSV file
def get_dataframe(filename: str ) -> pd.DataFrame:
    input_data = pd.read_csv(filename, sep=',' , names= csv_headers)
    input_data['timestamp']= pd.to_datetime(input_data['timestamp'])
    return input_data

# Get the tags for the filenames
def get_tags_dataframe():
    input_data = pd.read_csv('fall_scene_tag.csv', sep=',' , names= fall_scene_tags_headers)
    input_data = input_data.set_index('filename')
    return input_data

# Get the main info for a file
def file_info(df : pd.DataFrame)->():
    row_count = df.shape[0]
    total_time = df['timestamp'].max() - df['timestamp'].min()
    avg_delta = total_time / row_count
    fps = timedelta(seconds = 1) /avg_delta
    return row_count , total_time , avg_delta, fps
    
# plots a row of a dataframe
def plt_map(data):
    fig, axs = plt.subplots(1,2 ,figsize=(14, 5 ))
    absolute = data.values.copy()
    absolute.resize(4,4)
    mn = absolute.mean();
    
    # Main Plot (absolute temperatures)
    ax = sns.heatmap(absolute,linewidth=0, ax=axs[0], annot=True ,fmt='.1f')
    ax.set_title('Absolute temperatures (℃)')
    
    # Over mean plot 
    ax = sns.heatmap(absolute - mn > above_avg_threshold, linewidth=0, ax = axs[1])
    ax.set_title(f'Cells with temperature greater than {mn:0.2f} + '+ str(above_avg_threshold) +" ℃")
    plt.show()

In [3]:
# Load tags to identify the scene of the file and the event
tags_df = get_tags_dataframe()

In [4]:
# SELECT DIRECTORY
#cur_dir = fall_directory
cur_dir = non_fall_directory

# SELECT FILE
#cur_file = 'f_8.csv'
cur_file = 'nf_201.csv'

cur_df = get_dataframe(cur_dir + cur_file)
r_cnt , t_time , a_dta , fps = file_info(cur_df)

evt = tags_df.loc[cur_file, 'event']
scene = tags_df.loc[cur_file, 'scene']





print('FILE LOADED')
print('valid file:\t', cur_dir + cur_file)
print('event:\t', evt.upper())
print('scene:\t', scene.upper())
print('rows:\t', r_cnt)
print('total time:\t' , str(t_time))
print('average delta:\t' , a_dta)
print('fps:\t', fps)

FILE LOADED
valid file:	 ./non_fall_data/nf_201.csv
event:	 NOT FALL
scene:	 FRONTAL WALKING PICKING SCENE
rows:	 112
total time:	 0 days 00:00:11.222000
average delta:	 0 days 00:00:00.100196428
fps:	 9.980395708318065


In [5]:
# Re-run this cell if you select a different file
# Because the MAX property updates when you run this cell
# if MAX property isn't updated you probably won't see all the file
# or get an index out of bounds exception

@interact(sl_number = IntSlider(min=0 , max = r_cnt -1 , description = 'frame: ' ))
def explore_file(sl_number):
    data = cur_df[data_headers].iloc[sl_number]
    print('FRAME ', sl_number)
    print(f'MEAN: {data.mean(): 0.3f}')
    plt_map(data)

interactive(children=(IntSlider(value=0, description='frame: ', max=111), Output()), _dom_classes=('widget-int…